In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import gc
import torch
import pandas as pd
import numpy as np
import scipy.sparse

device = "cuda:9"

In [5]:
a = torch.rand(100000, 20000, device=device)

In [6]:
a = pd.read_csv('data/orig_data/evaluation_ids.csv')
a.head()

,row_id,cell_id,gene_id
0,0,c2150f55becb,CD86
1,1,c2150f55becb,CD274
2,2,c2150f55becb,CD270
3,3,c2150f55becb,CD155
4,4,c2150f55becb,CD112


In [8]:
print('evaluation_ids shape:', a.shape)
print('unique cell_id:', len(a['cell_id'].unique()))
print('unique gene_id:', len(a['gene_id'].unique()))


evaluation_ids shape: (65744180, 3)
unique cell_id: 65443
unique gene_id: 23558


In [9]:
b = pd.read_csv('data/orig_data/sample_submission.csv')
b.head()

,row_id,target
0,0,0.0
1,1,0.0
2,2,0.0
3,3,0.0
4,4,0.0


In [11]:
print('sample submission shape:', b.shape)
print('unique row_id:', len(b['row_id'].unique()))
print('unique target:', len(b['target'].unique()))

sample submission shape: (65744180, 2)
unique row_id: 65744180
unique target: 1


In [13]:
ap = pd.read_parquet('data/suzuki_data/evaluation_ids.parquet')
ap.head()

,row_id,cell_id,gene_id
0,0,c2150f55becb,CD86
1,1,c2150f55becb,CD274
2,2,c2150f55becb,CD270
3,3,c2150f55becb,CD155
4,4,c2150f55becb,CD112


In [14]:
print('evaluation_ids shape:', ap.shape)
print('unique cell_id:', len(ap['cell_id'].unique()))
print('unique gene_id:', len(ap['gene_id'].unique()))

evaluation_ids shape: (65744180, 3)
unique cell_id: 65443
unique gene_id: 23558


In [6]:
df = pd.read_parquet('data/suzuki_data/metadata.parquet')
df.head()

,cell_id,day,donor,cell_type,technology
0,c2150f55becb,2,27678,HSC,citeseq
1,65b7edf8a4da,2,27678,HSC,citeseq
2,c1b26cb1057b,2,27678,EryP,citeseq
3,917168fa6f83,2,27678,NeuP,citeseq
4,2b29feeca86d,2,27678,EryP,citeseq


In [5]:
print('metadata shape:', df.shape)
print('unique cell_id:', len(df['cell_id'].unique()))
print('unique day:', len(df['day'].unique()))
print('unique donor:', len(df['donor'].unique()))
print('unique cell_type:', len(df['cell_type'].unique()))
print('unique technology:', len(df['technology'].unique()))

metadata shape: (281528, 5)
unique cell_id: 281528
unique day: 5
unique donor: 4
unique cell_type: 8
unique technology: 2


In [15]:
df = pd.read_hdf('data/orig_data/train_cite_inputs.h5')
df.head()

gene_id,ENSG00000121410_A1BG,ENSG00000268895_A1BG-AS1,ENSG00000175899_A2M,ENSG00000245105_A2M-AS1,ENSG00000166535_A2ML1,ENSG00000128274_A4GALT,ENSG00000094914_AAAS,ENSG00000081760_AACS,ENSG00000109576_AADAT,ENSG00000103591_AAGAB,...,ENSG00000153975_ZUP1,ENSG00000086827_ZW10,ENSG00000174442_ZWILCH,ENSG00000122952_ZWINT,ENSG00000198205_ZXDA,ENSG00000198455_ZXDB,ENSG00000070476_ZXDC,ENSG00000162378_ZYG11B,ENSG00000159840_ZYX,ENSG00000074755_ZZEF1
cell_id,,,,,,,,,,,,,,,,,,,,,
45006fe3e4c8,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.00000,0.000000,4.090185,0.0
d02759a80ba2,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,...,0.000000,0.000000,0.000000,4.039545,0.0,0.0,0.00000,0.000000,0.000000,0.0
c016c6b0efa5,0.0,0.0,0.0,0.0,0.0,3.847321,0.000000,3.847321,3.847321,0.0,...,0.000000,0.000000,3.847321,4.529743,0.0,0.0,0.00000,3.847321,3.847321,0.0
ba7f733a4f75,0.0,0.0,0.0,0.0,0.0,0.000000,3.436846,3.436846,0.000000,0.0,...,3.436846,0.000000,4.113780,5.020215,0.0,0.0,0.00000,3.436846,4.113780,0.0
fbcf2443ffb2,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,4.196826,0.000000,0.0,...,0.000000,4.196826,4.196826,4.196826,0.0,0.0,3.51861,4.196826,3.518610,0.0


In [22]:
df = pd.read_hdf('data/orig_data/train_cite_targets.h5')
df.head()

gene_id,CD86,CD274,CD270,CD155,CD112,CD47,CD48,CD40,CD154,CD52,...,CD94,CD162,CD85j,CD23,CD328,HLA-E,CD82,CD101,CD88,CD224
cell_id,,,,,,,,,,,,,,,,,,,,,
45006fe3e4c8,1.167804,0.622530,0.106959,0.324989,3.331674,6.426002,1.480766,-0.728392,-0.468851,-0.073285,...,-0.448390,3.220174,-0.533004,0.674956,-0.006187,0.682148,1.398105,0.414292,1.780314,0.548070
d02759a80ba2,0.818970,0.506009,1.078682,6.848758,3.524885,5.279456,4.930438,2.069372,0.333652,-0.468088,...,0.323613,8.407108,0.131301,0.047607,-0.243628,0.547864,1.832587,0.982308,2.736507,2.184063
c016c6b0efa5,-0.356703,-0.422261,-0.824493,1.137495,0.518924,7.221962,-0.375034,1.738071,0.142919,-0.971460,...,1.348692,4.888579,-0.279483,-0.131097,-0.177604,-0.689188,9.013709,-1.182975,3.958148,2.868600
ba7f733a4f75,-1.201507,0.149115,2.022468,6.021595,7.258670,2.792436,21.708519,-0.137913,1.649969,-0.754680,...,1.504426,12.391979,0.511394,0.587863,-0.752638,1.714851,3.893782,1.799661,1.537249,4.407671
fbcf2443ffb2,-0.100404,0.697461,0.625836,-0.298404,1.369898,3.254521,-1.659380,0.643531,0.902710,1.291877,...,0.777023,6.496499,0.279898,-0.841950,-0.869419,0.675092,5.259685,-0.835379,9.631781,1.765445


In [27]:
df = pd.read_hdf('data/orig_data/train_multi_inputs.h5')
print("train_multi_inputs shape:", df.shape)
print(df.head())
del df
gc.collect()

train_multi_inputs shape: (105942, 228942)
gene_id       GL000194.1:114519-115365  GL000194.1:55758-56597  \
cell_id                                                          
56390cf1b95e                       0.0                     0.0   
fc0c60183c33                       0.0                     0.0   
9b4a87e22ad0                       0.0                     0.0   
81cccad8cd81                       0.0                     0.0   
15cb3d85c232                       0.0                     0.0   

gene_id       GL000194.1:58217-58957  GL000194.1:59535-60431  \
cell_id                                                        
56390cf1b95e                     0.0                     0.0   
fc0c60183c33                     0.0                     0.0   
9b4a87e22ad0                     0.0                     0.0   
81cccad8cd81                     0.0                     0.0   
15cb3d85c232                     0.0                     0.0   

gene_id       GL000195.1:119766-120427  GL000

1646

In [123]:
df = pd.read_hdf('data/orig_data/train_multi_targets.h5')
print("train_multi_targets shape:", df.shape)
df.head()

train_multi_targets shape: (105942, 23418)


gene_id,ENSG00000121410,ENSG00000268895,ENSG00000175899,ENSG00000245105,ENSG00000166535,ENSG00000256661,ENSG00000184389,ENSG00000128274,ENSG00000094914,ENSG00000081760,...,ENSG00000086827,ENSG00000174442,ENSG00000122952,ENSG00000198205,ENSG00000198455,ENSG00000070476,ENSG00000203995,ENSG00000162378,ENSG00000159840,ENSG00000074755
cell_id,,,,,,,,,,,,,,,,,,,,,
56390cf1b95e,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,4.893861,0.0,0.0,0.000000,0.0,5.583255,0.000000,4.893861
fc0c60183c33,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
9b4a87e22ad0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,5.107832,0.0,0.0,0.000000,0.0,0.000000,0.000000,5.107832
81cccad8cd81,0.0,4.507936,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5.195558,4.507936,0.0,0.0,0.000000,0.0,0.000000,0.000000,5.195558
15cb3d85c232,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.0,5.531572,0.0,0.000000,4.842377,0.000000


In [28]:
df = pd.read_hdf('data/orig_data/test_multi_inputs.h5')
print("test_multi_inputs shape:", df.shape)
df.head()

test_multi_inputs shape: (55935, 228942)


gene_id,GL000194.1:114519-115365,GL000194.1:55758-56597,GL000194.1:58217-58957,GL000194.1:59535-60431,GL000195.1:119766-120427,GL000195.1:120736-121603,GL000195.1:137437-138345,GL000195.1:15901-16653,GL000195.1:22357-23209,GL000195.1:23751-24619,...,chrY:7722278-7723128,chrY:7723971-7724880,chrY:7729854-7730772,chrY:7731785-7732664,chrY:7810142-7811040,chrY:7814107-7815018,chrY:7818751-7819626,chrY:7836768-7837671,chrY:7869454-7870371,chrY:7873814-7874709
cell_id,,,,,,,,,,,,,,,,,,,,,
458c2ae2c9b1,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01a0659b0710,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
028a8bc3f2ba,0.0,0.0,0.0,0.0,0.0,0.0,2.951019,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7ec0ca8bb863,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
caa0b0022cdc,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [85]:
multi_train_input_index_column = np.load("data/suzuki_data/train_multi_inputs_idxcol.npz", allow_pickle=True)
multi_train_input_index = multi_train_input_index_column["index"]
multi_input_columns = multi_train_input_index_column["columns"]
multi_train_values = scipy.sparse.load_npz("data/suzuki_data/train_multi_inputs_values.sparse.npz")

In [91]:
multi_train_input_index.shape, multi_train_input_index[:3]

((105942,),
 array(['56390cf1b95e', 'fc0c60183c33', '9b4a87e22ad0'], dtype=object))

In [90]:
multi_input_columns.shape, multi_input_columns[:3]

((228942,),
 array(['GL000194.1:114519-115365', 'GL000194.1:55758-56597',
        'GL000194.1:58217-58957'], dtype=object))

In [33]:
multi_input_columns[0][0:multi_input_columns[0].find(":")]

'GL000194.1'

In [39]:
len(multi_train_values.data), len(multi_train_input_index)

(607301546, 105942)

In [52]:
multi_train_values[0, 0:3].toarray().ravel()

array([0., 0., 0.], dtype=float32)

In [45]:
len(multi_train_values.data[multi_train_values.indptr[1] : multi_train_values.indptr[1 + 1]])

2733

In [58]:
df = pd.read_parquet('data/suzuki_data/multi_cell_statistics.parquet')
print(df.shape)
df.head()

(161877, 43)


,nonzero_ratio,nonzero_q25,nonzero_q50,nonzero_q75,mean,std,ch_nonzero_ratio_GL000194.1,ch_nonzero_ratio_GL000195.1,ch_nonzero_ratio_GL000205.2,ch_nonzero_ratio_GL000218.1,...,ch_nonzero_ratio_chr2,ch_nonzero_ratio_chr3,ch_nonzero_ratio_chr4,ch_nonzero_ratio_chr5,ch_nonzero_ratio_chr6,ch_nonzero_ratio_chr7,ch_nonzero_ratio_chr8,ch_nonzero_ratio_chr9,ch_nonzero_ratio_chrX,ch_nonzero_ratio_chrY
56390cf1b95e,0.022784,1.554948,2.525614,3.641098,0.062538,0.454389,0.0,0.0,0.0,0.0,...,0.019935,0.020821,0.018709,0.021942,0.021679,0.020824,0.021011,0.024617,0.018175,0.023096
fc0c60183c33,0.011867,2.041723,3.009425,4.469999,0.040418,0.407716,0.0,0.0,0.0,0.0,...,0.009550,0.010372,0.008329,0.008335,0.011649,0.011457,0.009127,0.012331,0.007116,0.009302
9b4a87e22ad0,0.020745,1.640358,2.605274,3.802436,0.059589,0.454111,0.0,0.0,0.0,0.0,...,0.020192,0.018678,0.018621,0.017560,0.018702,0.020169,0.017224,0.021691,0.016420,0.013921
81cccad8cd81,0.021857,1.606161,2.472519,3.585912,0.059788,0.442295,0.0,0.0,0.0,0.0,...,0.018341,0.019230,0.019238,0.018757,0.019695,0.020987,0.017603,0.021481,0.016261,0.009302
15cb3d85c232,0.019641,1.654128,2.594813,3.801200,0.056526,0.444164,0.0,0.0,0.0,0.0,...,0.017620,0.018433,0.017737,0.016921,0.018133,0.019514,0.017413,0.020329,0.012581,0.013921


In [82]:
df # cite_batch_inputs.parquet

,batch_sv0,batch_sv1,batch_sv2,batch_sv3,batch_sv4,batch_sv5,batch_sv6,batch_sv7
group,,,,,,,,
0,94.716789,-13.627334,1.010133,-7.014981,5.242704,2.608015,1.241862,0.494505
1,98.997696,0.751707,1.241592,4.302948,-8.720427,7.466218,-5.442671,-3.878374
2,97.084274,1.885003,2.955458,0.824604,-2.067913,2.420853,-4.916955,-8.079290
3,96.656441,12.271877,11.828243,-2.720859,-1.936250,0.804851,0.440720,-4.025300
5,94.716789,-13.627334,1.010133,-7.014981,5.242704,2.608015,1.241862,0.494505
6,99.065826,-6.025971,-2.552545,8.351398,-0.153087,3.618849,7.699559,1.284394
7,101.877632,4.600351,-5.903766,7.208352,12.268522,5.772725,-1.599355,1.007039
8,99.680504,12.809192,8.564008,-0.202896,5.840864,2.251115,5.407852,-0.524744
10,93.237091,-12.224535,2.842484,-5.382669,-1.820358,-3.725021,-3.743368,1.230684


In [84]:
normalized_df  # normalized_cite_batch_inputs.parquet

,batch_sv0,batch_sv1,batch_sv2,batch_sv3,batch_sv4,batch_sv5,batch_sv6,batch_sv7
group,,,,,,,,
0,-1.022774,-1.367788,0.116033,-1.104394,0.934371,0.525153,0.262365,0.108675
1,0.124417,0.100759,0.148325,0.684701,-1.555540,1.504638,-1.124791,-0.843172
2,-0.388339,0.216504,0.387434,0.134858,-0.369261,0.487418,-1.015696,-1.757588
3,-0.502989,1.277326,1.625318,-0.425595,-0.345783,0.161609,0.096114,-0.875154
5,-1.022774,-1.367788,0.116033,-1.104394,0.934371,0.525153,0.262365,0.108675
6,0.142674,-0.591453,-0.381013,1.324665,-0.027809,0.728952,1.602449,0.280610
7,0.896178,0.493825,-0.848558,1.143976,2.187217,1.163205,-0.327237,0.220238
8,0.307395,1.332202,1.169909,-0.027565,1.041035,0.453197,1.126880,-0.113186
10,-1.419302,-1.224518,0.371673,-0.846364,-0.325117,-0.751680,-0.772157,0.268919


In [97]:
df = pd.read_table('data/suzuki_data/hgnc_complete_set.txt')
df.head()

/tmp/ipykernel_3739063/3166818857.py:1: DtypeWarning: Columns (32,34,38,40,50) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_table('data/suzuki_data/hgnc_complete_set.txt')


,hgnc_id,symbol,name,locus_group,locus_type,status,location,location_sortable,alias_symbol,alias_name,...,cd,lncrnadb,enzyme_id,intermediate_filament_db,rna_central_id,lncipedia,gtrnadb,agr,mane_select,gencc
0,HGNC:5,A1BG,alpha-1-B glycoprotein,protein-coding gene,gene with protein product,Approved,19q13.43,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HGNC:5,ENST00000263100.8|NM_130786.4,NaN
1,HGNC:37133,A1BG-AS1,A1BG antisense RNA 1,non-coding RNA,"RNA, long non-coding",Approved,19q13.43,NaN,FLJ23569,NaN,...,NaN,NaN,NaN,NaN,URS00007E4F6E,A1BG-AS1,NaN,HGNC:37133,NaN,NaN
2,HGNC:24086,A1CF,APOBEC1 complementation factor,protein-coding gene,gene with protein product,Approved,10q11.23,NaN,ACF|ASP|ACF64|ACF65|APOBEC1CF,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HGNC:24086,ENST00000373997.8|NM_014576.4,NaN
3,HGNC:7,A2M,alpha-2-macroglobulin,protein-coding gene,gene with protein product,Approved,12p13.31,NaN,FWP007|S863-7|CPAMD5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HGNC:7,ENST00000318602.12|NM_000014.6,HGNC:7
4,HGNC:27057,A2M-AS1,A2M antisense RNA 1,non-coding RNA,"RNA, long non-coding",Approved,12p13.31,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,URS00001F234A,A2M-AS1,NaN,HGNC:27057,NaN,NaN


In [108]:
def read_reactome_gmt(file_path):
    data_dict_list = []
    with open(file_path) as f:
        for line in f:
            values = line.strip().split("\t")
            st_id = values[1]
            genes = values[3:]
            for gene in genes:
                data_dict_list.append({"st_id": st_id, "gene": gene})
    df = pd.DataFrame(data_dict_list)
    return df
pathway_gene_df = read_reactome_gmt('data/suzuki_data/ReactomePathways.gmt')
print(pathway_gene_df.shape)
pathway_gene_df.head()

(128927, 2)


,st_id,gene
0,R-HSA-164843,HMGA1
1,R-HSA-164843,LIG4
2,R-HSA-164843,PSIP1
3,R-HSA-164843,XRCC4
4,R-HSA-164843,XRCC5


In [109]:
inputs_values = scipy.sparse.load_npz("data/suzuki_data/train_cite_inputs_values.sparse.npz")
inputs_values.shape

(70988, 22050)

In [122]:
a = np.array([True]+[False]*70987)
inputs_values[a, :]

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 3824 stored elements and shape (1, 22050)>

In [111]:
targets_values = scipy.sparse.load_npz("data/suzuki_data/train_cite_targets_values.sparse.npz")
targets_values = targets_values.toarray()
targets_values.shape

(70988, 140)

In [6]:
cell_statistics = pd.read_parquet('data/suzuki_data/cite_cell_statistics.parquet')
print(cell_statistics.shape)
cell_statistics.head()

(119651, 6)


,nonzero_ratio,nonzero_q25,nonzero_q50,nonzero_q75,mean,std
45006fe3e4c8,0.173424,4.090185,4.090185,5.177577,0.821081,1.834528
d02759a80ba2,0.168163,4.039545,4.039545,5.126350,0.792177,1.805970
c016c6b0efa5,0.209161,3.847321,3.847321,4.931607,0.940229,1.875602
ba7f733a4f75,0.266712,3.436846,4.113780,4.798720,1.135173,1.951249
fbcf2443ffb2,0.264898,3.518610,4.196826,4.882423,1.155327,1.988160


In [7]:
metadata = pd.read_parquet('data/suzuki_data/metadata.parquet')
print(metadata.shape)
metadata.head()

(281528, 5)


,cell_id,day,donor,cell_type,technology
0,c2150f55becb,2,27678,HSC,citeseq
1,65b7edf8a4da,2,27678,HSC,citeseq
2,c1b26cb1057b,2,27678,EryP,citeseq
3,917168fa6f83,2,27678,NeuP,citeseq
4,2b29feeca86d,2,27678,EryP,citeseq


In [5]:
df = pd.read_parquet('data/suzuki_data/normalized_cite_batch_inputs.parquet')
print(df.shape)
df

(16, 8)


,batch_sv0,batch_sv1,batch_sv2,batch_sv3,batch_sv4,batch_sv5,batch_sv6,batch_sv7
group,,,,,,,,
0,-1.022774,-1.367788,0.116033,-1.104394,0.934371,0.525153,0.262365,0.108675
1,0.124417,0.100759,0.148325,0.684701,-1.555540,1.504638,-1.124791,-0.843172
2,-0.388339,0.216504,0.387434,0.134858,-0.369261,0.487418,-1.015696,-1.757588
3,-0.502989,1.277326,1.625318,-0.425595,-0.345783,0.161609,0.096114,-0.875154
5,-1.022774,-1.367788,0.116033,-1.104394,0.934371,0.525153,0.262365,0.108675
6,0.142674,-0.591453,-0.381013,1.324665,-0.027809,0.728952,1.602449,0.280610
7,0.896178,0.493825,-0.848558,1.143976,2.187217,1.163205,-0.327237,0.220238
8,0.307395,1.332202,1.169909,-0.027565,1.041035,0.453197,1.126880,-0.113186
10,-1.419302,-1.224518,0.371673,-0.846364,-0.325117,-0.751680,-0.772157,0.268919


In [9]:
import pandas as pd

# Left DataFrame with a column to join on
left_df = pd.DataFrame({
    'ID': [1, 2, 3, 4],
    'Name': ['Alice', 'Bob', 'Charlie', 'David']
})

# Right DataFrame with an index to join on
right_df = pd.DataFrame({
    'Score': [85, 90, 78, 92]
}, index=[4, 2, 5, 9])

# Merge on left_df['ID'] and right_df's index
merged_df = pd.merge(left_df, right_df, left_on='ID', right_index=True, how='left')
print(merged_df)

   ID     Name  Score
0   1    Alice    NaN
1   2      Bob   90.0
2   3  Charlie    NaN
3   4    David   85.0


# Multi ========================================================================

In [1]:
import os
import scipy.sparse
from utils.tools import *
import numpy as np
data_dir = "data/suzuki_data"

##### train_multi_inputs_values ------------------------------------------------

In [2]:
inputs_path = "train_multi_inputs_values.sparse.npz"
train_multi_inputs_values = scipy.sparse.load_npz(os.path.join(data_dir, inputs_path))
train_multi_inputs_values

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 607301546 stored elements and shape (105942, 228942)>

In [5]:
# first_two_rows = train_multi_inputs_values[:2,:]
# output_path = "data/fast_suzuki_data/train_multi_inputs_values.sparse.npz"
# scipy.sparse.save_npz(output_path, first_two_rows)

##### train multi metadata_df --------------------------------------------------

In [6]:
input_index = np.load(os.path.join(data_dir, "train_multi_inputs_idxcol.npz"), allow_pickle=True)["index"]
metadata_df = pd.read_parquet(os.path.join(data_dir, "metadata.parquet"))
metadata_df = metadata_df.set_index("cell_id")
metadata_df = metadata_df.loc[input_index, :]
cell_statistics_df = pd.read_parquet(os.path.join(data_dir, "normalized_multi_cell_statistics.parquet"))
metadata_df = pd.merge(metadata_df, cell_statistics_df, left_index=True, right_index=True)
metadata_df["group"] = get_group_id(metadata_df)
batch_statistics_df = pd.read_parquet(os.path.join(data_dir, "normalized_multi_batch_statistics.parquet"))
metadata_df = pd.merge(metadata_df, batch_statistics_df, left_on="group", right_index=True)
print(metadata_df.shape)
metadata_df.head()

(105942, 57)


,day,donor,cell_type,technology,nonzero_ratio,nonzero_q25,nonzero_q50,nonzero_q75,mean,std,...,group,cell_ratio_hidden,cell_count,cell_ratio_HSC,cell_ratio_NeuP,cell_ratio_EryP,cell_ratio_MkP,cell_ratio_MasP,cell_ratio_BP,cell_ratio_MoP
56390cf1b95e,2,32606,NeuP,multiome,-0.153827,0.401305,0.431351,0.189615,0.987372,1.682855,...,25,-0.743392,-0.91482,1.212595,0.36879,0.748816,0.6308,0.050347,1.407156,-0.4825
fc0c60183c33,2,32606,HSC,multiome,-0.784652,1.287659,0.982307,0.774217,-0.629910,0.734579,...,25,-0.743392,-0.91482,1.212595,0.36879,0.748816,0.6308,0.050347,1.407156,-0.4825
9b4a87e22ad0,2,32606,MasP,multiome,-0.271629,0.556827,0.522067,0.303403,0.771758,1.677193,...,25,-0.743392,-0.91482,1.212595,0.36879,0.748816,0.6308,0.050347,1.407156,-0.4825
81cccad8cd81,2,32606,HSC,multiome,-0.207389,0.494558,0.370887,0.150694,0.786278,1.437130,...,25,-0.743392,-0.91482,1.212595,0.36879,0.748816,0.6308,0.050347,1.407156,-0.4825
15cb3d85c232,2,32606,MkP,multiome,-0.335447,0.581900,0.510154,0.302531,0.547776,1.475110,...,25,-0.743392,-0.91482,1.212595,0.36879,0.748816,0.6308,0.050347,1.407156,-0.4825


In [7]:
# first_two_rows = metadata_df.iloc[:2]
# output_path = "data/fast_suzuki_data/train_multi_metadata.parquet"
# first_two_rows.to_parquet(output_path)

##### train_multi_targets_values -----------------------------------------------

In [8]:
targets_path = "train_multi_targets_values.sparse.npz"
train_multi_targets_values = scipy.sparse.load_npz(os.path.join(data_dir, targets_path))
train_multi_targets_values

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 407024875 stored elements and shape (105942, 23418)>

In [9]:
# first_two_rows = train_multi_targets_values[:2,:]
# output_path = "data/fast_suzuki_data/train_multi_targets_values.sparse.npz"
# scipy.sparse.save_npz(output_path, first_two_rows)

##### test_multi_inputs_values -------------------------------------------------

In [10]:
inputs_path = "test_multi_inputs_values.sparse.npz"
test_multi_inputs_values = scipy.sparse.load_npz(os.path.join(data_dir, inputs_path))
test_multi_inputs_values

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 353572710 stored elements and shape (55935, 228942)>

In [11]:
# first_two_rows = test_multi_inputs_values[:2,:]
# output_path = "data/fast_suzuki_data/test_multi_inputs_values.sparse.npz"
# scipy.sparse.save_npz(output_path, first_two_rows)

##### test multi metadata_df ---------------------------------------------------

In [12]:
input_index = np.load(os.path.join(data_dir, "test_multi_inputs_idxcol.npz"), allow_pickle=True)["index"]
metadata_df = pd.read_parquet(os.path.join(data_dir, "metadata.parquet"))
metadata_df = metadata_df.set_index("cell_id")
metadata_df = metadata_df.loc[input_index, :]
cell_statistics_df = pd.read_parquet(os.path.join(data_dir, "normalized_multi_cell_statistics.parquet"))
metadata_df = pd.merge(metadata_df, cell_statistics_df, left_index=True, right_index=True)
metadata_df["group"] = get_group_id(metadata_df)
batch_statistics_df = pd.read_parquet(os.path.join(data_dir, "normalized_multi_batch_statistics.parquet"))
metadata_df = pd.merge(metadata_df, batch_statistics_df, left_on="group", right_index=True)
print(metadata_df.shape)
metadata_df.head()

(55935, 57)


,day,donor,cell_type,technology,nonzero_ratio,nonzero_q25,nonzero_q50,nonzero_q75,mean,std,...,group,cell_ratio_hidden,cell_count,cell_ratio_HSC,cell_ratio_NeuP,cell_ratio_EryP,cell_ratio_MkP,cell_ratio_MasP,cell_ratio_BP,cell_ratio_MoP
458c2ae2c9b1,2,27678,hidden,multiome,-0.702650,1.104078,0.972525,0.791557,-0.309180,1.216415,...,20,1.274386,0.292224,-1.115352,-1.231387,-1.230523,-1.181566,-1.042193,-0.829681,-0.877258
01a0659b0710,2,27678,hidden,multiome,0.110048,-0.185325,-0.415898,-0.482598,0.597141,0.189259,...,20,1.274386,0.292224,-1.115352,-1.231387,-1.230523,-1.181566,-1.042193,-0.829681,-0.877258
028a8bc3f2ba,2,27678,hidden,multiome,-0.047591,0.053240,-0.090235,-0.230203,0.629066,0.644627,...,20,1.274386,0.292224,-1.115352,-1.231387,-1.230523,-1.181566,-1.042193,-0.829681,-0.877258
7ec0ca8bb863,2,27678,hidden,multiome,0.197412,-0.181654,-0.319543,-0.444943,0.890877,0.489352,...,20,1.274386,0.292224,-1.115352,-1.231387,-1.230523,-1.181566,-1.042193,-0.829681,-0.877258
caa0b0022cdc,2,27678,hidden,multiome,0.013696,0.245883,0.296361,0.009914,1.232613,1.576152,...,20,1.274386,0.292224,-1.115352,-1.231387,-1.230523,-1.181566,-1.042193,-0.829681,-0.877258


In [13]:
# first_two_rows = metadata_df.iloc[:2]
# output_path = "data/fast_suzuki_data/test_multi_metadata.parquet"
# first_two_rows.to_parquet(output_path)

##### test mutli target --> None 不存在------------------------------------------

# Cite =========================================================================

##### train_cite_inputs_values ------------------------------------------------

In [14]:
inputs_path = "train_cite_inputs_values.sparse.npz"
train_cite_inputs_values = scipy.sparse.load_npz(os.path.join(data_dir, inputs_path))
train_cite_inputs_values

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 344303478 stored elements and shape (70988, 22050)>

In [15]:
# first_two_rows = train_cite_inputs_values[:2,:]
# output_path = "data/fast_suzuki_data/train_cite_inputs_values.sparse.npz"
# scipy.sparse.save_npz(output_path, first_two_rows)

##### train cite metadata_df ---------------------------------------------------

In [16]:
input_index = np.load(os.path.join(data_dir, "train_cite_inputs_idxcol.npz"), allow_pickle=True)["index"]
metadata_df = pd.read_parquet(os.path.join(data_dir, "metadata.parquet"))
metadata_df = metadata_df.set_index("cell_id")
metadata_df = metadata_df.loc[input_index, :]
cell_statistics_df = pd.read_parquet(os.path.join(data_dir, "normalized_cite_cell_statistics.parquet"))
metadata_df = pd.merge(metadata_df, cell_statistics_df, left_index=True, right_index=True)
metadata_df["group"] = get_group_id(metadata_df)
batch_statistics_df = pd.read_parquet(os.path.join(data_dir, "normalized_cite_batch_statistics.parquet"))
metadata_df = pd.merge(metadata_df, batch_statistics_df, left_on="group", right_index=True)
batch_inputs_df = pd.read_parquet(os.path.join(data_dir, "normalized_cite_batch_inputs.parquet"))
metadata_df = pd.merge(metadata_df, batch_inputs_df, left_on="group", right_index=True)
print(metadata_df.shape)
metadata_df.head()

(70988, 27)


,day,donor,cell_type,technology,nonzero_ratio,nonzero_q25,nonzero_q50,nonzero_q75,mean,std,...,cell_ratio_MoP,cell_count,batch_sv0,batch_sv1,batch_sv2,batch_sv3,batch_sv4,batch_sv5,batch_sv6,batch_sv7
45006fe3e4c8,2,32606,HSC,citeseq,-0.967283,0.783625,-0.308014,1.286971,-0.943154,-0.698494,...,-0.910177,-0.001836,-1.022774,-1.367788,0.116033,-1.104394,0.934371,0.525153,0.262365,0.108675
d02759a80ba2,2,32606,HSC,citeseq,-1.077987,0.649684,-0.509828,1.059072,-1.121040,-1.082617,...,-0.910177,-0.001836,-1.022774,-1.367788,0.116033,-1.104394,0.934371,0.525153,0.262365,0.108675
c016c6b0efa5,2,32606,EryP,citeseq,-0.215266,0.141268,-1.275883,0.192697,-0.209878,-0.146027,...,-0.910177,-0.001836,-1.022774,-1.367788,0.116033,-1.104394,0.934371,0.525153,0.262365,0.108675
ba7f733a4f75,2,32606,NeuP,citeseq,0.995786,-0.944409,-0.213983,-0.398487,0.989871,0.871464,...,-0.910177,-0.001836,-1.022774,-1.367788,0.116033,-1.104394,0.934371,0.525153,0.262365,0.108675
fbcf2443ffb2,2,32606,EryP,citeseq,0.957613,-0.728150,0.116976,-0.026109,1.113905,1.367942,...,-0.910177,-0.001836,-1.022774,-1.367788,0.116033,-1.104394,0.934371,0.525153,0.262365,0.108675


In [17]:
# first_two_rows = metadata_df.iloc[:2]
# output_path = "data/fast_suzuki_data/train_cite_metadata.parquet"
# first_two_rows.to_parquet(output_path)

##### train_cite_targets_values -----------------------------------------------

In [18]:
targets_path = "train_cite_targets_values.sparse.npz"
train_cite_targets_values = scipy.sparse.load_npz(os.path.join(data_dir, targets_path))
train_cite_targets_values

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 9938320 stored elements and shape (70988, 140)>

In [19]:
# first_two_rows = train_cite_targets_values[:2,:]
# output_path = "data/fast_suzuki_data/train_cite_targets_values.sparse.npz"
# scipy.sparse.save_npz(output_path, first_two_rows)

##### test_cite_inputs_values -------------------------------------------------

In [20]:
inputs_path = "test_cite_inputs_values.sparse.npz"
test_cite_inputs_values = scipy.sparse.load_npz(os.path.join(data_dir, inputs_path))
test_cite_inputs_values

<Compressed Sparse Row sparse matrix of dtype 'float32'
	with 234516203 stored elements and shape (48663, 22050)>

In [21]:
# first_two_rows = test_cite_inputs_values[:2,:]
# output_path = "data/fast_suzuki_data/test_cite_inputs_values.sparse.npz"
# scipy.sparse.save_npz(output_path, first_two_rows)

##### test cite metadata_df ---------------------------------------------------

In [23]:
input_index = np.load(os.path.join(data_dir, "test_cite_inputs_idxcol.npz"), allow_pickle=True)["index"]
metadata_df = pd.read_parquet(os.path.join(data_dir, "metadata.parquet"))
metadata_df = metadata_df.set_index("cell_id")
metadata_df = metadata_df.loc[input_index, :]
cell_statistics_df = pd.read_parquet(os.path.join(data_dir, "normalized_cite_cell_statistics.parquet"))
metadata_df = pd.merge(metadata_df, cell_statistics_df, left_index=True, right_index=True)
metadata_df["group"] = get_group_id(metadata_df)
batch_statistics_df = pd.read_parquet(os.path.join(data_dir, "normalized_cite_batch_statistics.parquet"))
metadata_df = pd.merge(metadata_df, batch_statistics_df, left_on="group", right_index=True)
batch_inputs_df = pd.read_parquet(os.path.join(data_dir, "normalized_cite_batch_inputs.parquet"))
metadata_df = pd.merge(metadata_df, batch_inputs_df, left_on="group", right_index=True)
print(metadata_df.shape)
metadata_df.head()

(48663, 27)


,day,donor,cell_type,technology,nonzero_ratio,nonzero_q25,nonzero_q50,nonzero_q75,mean,std,...,cell_ratio_MoP,cell_count,batch_sv0,batch_sv1,batch_sv2,batch_sv3,batch_sv4,batch_sv5,batch_sv6,batch_sv7
c2150f55becb,2,27678,HSC,citeseq,-0.967283,0.783625,-0.308014,1.286971,-0.943154,-0.698494,...,-0.910177,-0.001836,-1.022774,-1.367788,0.116033,-1.104394,0.934371,0.525153,0.262365,0.108675
65b7edf8a4da,2,27678,HSC,citeseq,-1.077987,0.649684,-0.509828,1.059072,-1.121040,-1.082617,...,-0.910177,-0.001836,-1.022774,-1.367788,0.116033,-1.104394,0.934371,0.525153,0.262365,0.108675
c1b26cb1057b,2,27678,EryP,citeseq,-0.215266,0.141268,-1.275883,0.192697,-0.209878,-0.146027,...,-0.910177,-0.001836,-1.022774,-1.367788,0.116033,-1.104394,0.934371,0.525153,0.262365,0.108675
917168fa6f83,2,27678,NeuP,citeseq,0.995786,-0.944409,-0.213983,-0.398487,0.989871,0.871464,...,-0.910177,-0.001836,-1.022774,-1.367788,0.116033,-1.104394,0.934371,0.525153,0.262365,0.108675
2b29feeca86d,2,27678,EryP,citeseq,0.957613,-0.728150,0.116976,-0.026109,1.113905,1.367942,...,-0.910177,-0.001836,-1.022774,-1.367788,0.116033,-1.104394,0.934371,0.525153,0.262365,0.108675


In [24]:
# first_two_rows = metadata_df.iloc[:2]
# output_path = "data/fast_suzuki_data/test_cite_metadata.parquet"
# first_two_rows.to_parquet(output_path)

##### test cite target --> None 不存在------------------------------------------

In [36]:
import joblib
import numpy as np

loaded_svd = joblib.load('data/fast_preprocess/multi_targets_decomposer.joblib')
loaded_data = np.load('data/fast_preprocess/multi_targets_global_median.npy')
print(np.array(loaded_data).shape)

(23418,)
